In [1]:
import pandas as pd
from requests import get
from bs4 import BeautifulSoup
import requests
import bs4
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import TimeoutException, WebDriverException
import time
chrome_options = Options()
chrome_options.add_argument('--headless')# servers don't provide the visulazation
chrome_options.add_argument('--no-sandbox')# operate at the highest authority
chrome_options.add_argument('--disable-dev-shm-usage')#increase the RAM of chrome to load the page
path = "/Users/shubhamgoyanka/Desktop/BAIM 2021-22/Fall Mod 2/MGMT 590 - WDA/Lecture Notebooks/chromedriver"

In [2]:
citiesLink = ['https://chicago.craigslist.org/d/jobs/search/jjj', 'https://newyork.craigslist.org/d/jobs/search/jjj',
              'https://newjersey.craigslist.org/d/jobs/search/jjj', 'https://indianapolis.craigslist.org/d/jobs/search/jjj',
             'https://columbus.craigslist.org/d/jobs/search/jjj', 'https://sfbay.craigslist.org/d/jobs/search/jjj',
              'https://seattle.craigslist.org/d/jobs/search/jjj','https://austin.craigslist.org/d/jobs/search/jjj',
             'https://houston.craigslist.org/d/jobs/search/jjj', 'https://lasvegas.craigslist.org/d/jobs/search/jjj']

def getjobLinks(citiesLink):
    try:
        cityjoblink = {}
        for link in citiesLink:
            city = link[8:link.find('.')]
            cityjoblink[city] = []
            response = get(link)
            soup = BeautifulSoup(response.text, 'html.parser')
            for a in soup.find_all("a", class_="result-title hdrlnk"):
                cityjoblink[city].append(a.get('href'))
        return cityjoblink

    except Exception as e:
            print(e)

In [3]:
def getjobData():
    try:
        driver = webdriver.Chrome(path, options = chrome_options)
        cityjoblinkDict = getjobLinks(citiesLink)
        cityList=[]
        linkList=[]
        titleList=[]
        descriptionList=[]

        for key in cityjoblinkDict:
            jobLinks = cityjoblinkDict[key]
            for link in jobLinks:
                time.sleep(2)
                try:
                    response = get(link)
                    #print(link)
                    driver.get(link)
                    title = driver.find_element_by_class_name('postingtitletext').text.strip()
                    description = driver.find_element_by_xpath('//html/body/section/section/section/section').text.strip()
                    cityList.append(key)
                    linkList.append(link)
                    titleList.append(title)
                    descriptionList.append(description)
                except Exception as e:
                    print(e)
                finally:
                    continue
        driver.quit()
        return cityList, linkList, titleList, descriptionList
    except Exception as e:
        print(e)

In [4]:
cityList, linkList, titleList, descriptionList = getjobData()
df = pd.DataFrame({'location': cityList,'URL': linkList,'title':titleList, 'description': descriptionList})

<ipython-input-3-5ffd889056e7>:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(path, options = chrome_options)
<ipython-input-3-5ffd889056e7>:18: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  title = driver.find_element_by_class_name('postingtitletext').text.strip()
<ipython-input-3-5ffd889056e7>:19: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  description = driver.find_element_by_xpath('//html/body/section/section/section/section').text.strip()


In [5]:
df

,location,URL,title,description
0,chicago,https://chicago.craigslist.org/chc/fbh/d/chica...,Food Runners Wanted (Chicago),Sapori Trattoria located at 2701 N Halsted is ...
1,chicago,https://chicago.craigslist.org/nwc/trp/d/elgin...,CDL Class A truck driver (Regional-flatbed tra...,SCS Transport is hiring experienced and entry ...
2,chicago,https://chicago.craigslist.org/nwc/trp/d/addis...,CDL A DRIVER REGIONAL OR OTR (Addison),Compensation: $1750.00 WEEKLY (Monday thru Fri...
3,chicago,https://chicago.craigslist.org/nwc/lab/d/arlin...,Delivery Driver,Get $200 bonus for 20 orders within 20 days of...
4,chicago,https://chicago.craigslist.org/nwc/fbh/d/arlin...,Delivery Driver,Get $200 bonus for 20 orders within 20 days of...
...,...,...,...,...
1195,lasvegas,https://lasvegas.craigslist.org/lab/d/las-vega...,$10-$40 HOURLY Automotive Smog Technician 1G a...,Must Be Experienced\n\nMust have 1G or 2G Lice...
1196,lasvegas,https://lasvegas.craigslist.org/lab/d/las-vega...,$10-$30 HOURLY Automotive Technicians,Must be experienced.\n\nOpenings all over Las ...
1197,lasvegas,https://lasvegas.craigslist.org/lab/d/las-vega...,50K- 150K ANNUAL PAY Automotive Repair Shop Ma...,"Must be Experienced, and Must be People and Sa..."
1198,lasvegas,https://lasvegas.craigslist.org/trd/d/blue-dia...,Generator Technician - Hiring NOW $$$ (Las Vegas),Generator Service Technician is needed in the ...


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1200 entries, 0 to 1199
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   location     1200 non-null   object
 1   URL          1200 non-null   object
 2   title        1200 non-null   object
 3   description  1200 non-null   object
dtypes: object(4)
memory usage: 37.6+ KB


In [7]:
df['fradulent'] = ''
df.to_csv('scraped_data.csv', index = False)